In [1]:
import urllib
import requests
import json
import os
import re
import pyquery
import pandas as pd
import pickle
import numpy as np

In [2]:
def get_web_html(url):
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        else:    
            print('Wrong：', response.status_code)        
    except Exception as e:
        print(e)
        return None

In [3]:
def get_twelve_post(edges):   
    global all_posts
    global post_cnt
    global post_df
    global post_comments
    #12 post of this for loop
    for edge in edges:
        ps_info = {}
        ps_info['post_id'] = post_cnt
        if len(edge['node']['edge_media_to_caption']['edges']) > 0:
            ps_info['post_text'] = edge['node']['edge_media_to_caption']['edges'][0]['node']['text']
        else:
            ps_info['post_text'] = ''
        ps_info['post_coment_count'] = edge['node']['edge_media_to_comment']['count']
        ps_info['post_likes'] = edge['node']['edge_media_preview_like']['count']
        ps_info['post_time']=str(pd.to_datetime(edge['node']['taken_at_timestamp'],unit='s'))
        
        shortcode = edge['node']['shortcode']
        url_shortcode = 'https://www.instagram.com/p/'+shortcode+'/?__a=1'
        with urllib.request.urlopen(url_shortcode) as temp_u:
            js_data = json.loads(temp_u.read().decode())
        photo_in_post_cnt=0
        post = []
        for row in js_data['graphql']['shortcode_media']['edge_media_to_parent_comment']['edges']:
            info = {}
            info['text'] = row['node']['text']
            info['username'] = row['node']['owner']['username']  
            post_comments.append(info)
       
        ps_info['comments'] = post_comments
        post_df.append(ps_info)
        if 'edge_sidecar_to_children' in js_data['graphql']['shortcode_media']:
            edges_shortcode = js_data['graphql']['shortcode_media']['edge_sidecar_to_children']['edges']
            #num of pic in single post of this for loop            
            k = 1
            for edge_s in edges_shortcode:
                fn = 'post_'+str(post_cnt)+'_'+str(k)
                k += 1
                if edge_s['node']['is_video'] and edge_s['node']['video_url'] != 'https://static.cdninstagram.com/rsrc.php/null.jpg':
                    display_url = edge_s['node']['video_url']
                elif edge_s['node']['display_url']:
                    display_url = edge_s['node']['display_url']
                post.append([display_url,fn])
                photo_in_post_cnt=photo_in_post_cnt+1  
        else:
            fn = 'post_'+str(post_cnt)+'_1'
            if js_data['graphql']['shortcode_media']['is_video'] and js_data['graphql']['shortcode_media']['video_url'] != 'https://static.cdninstagram.com/rsrc.php/null.jpg':
                display_url = js_data['graphql']['shortcode_media']['video_url']
            elif js_data['graphql']['shortcode_media']['display_url']:
                display_url = js_data['graphql']['shortcode_media']['display_url']
            post.append([display_url,fn])
        all_posts.append(post)
        post_cnt=post_cnt+1

In [7]:
all_posts = []
post_cnt=1
post_comments = []
post_df = []

user_name = 'nasa'
#download path
max_post_iter=50

url = 'https://www.instagram.com/' + user_name
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
}
html = get_web_html(url)
user_id = re.findall('profilePage_([0-9]+)', html, re.S)[0]
doc = pyquery.PyQuery(html)
items = doc('script[type="text/javascript"]').items()

print("user_id:",user_id)
print("url:",url)

for item in items:
    if item.text().strip().startswith('window._sharedData'):
        js_data = json.loads(item.text()[21:-1], encoding='utf-8')
        edges = js_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['edges']
        cursor = js_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        flag = js_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
        get_twelve_post(edges)
        
for index in range(0,max_post_iter):
    if (flag):
        url_next = 'https://instagram.com/graphql/query/?query_id=17888483320059182&id='+user_id+'&first=12&after='+cursor
        print('url number: ',index)
        with urllib.request.urlopen(url_next) as temp_u:
            js_data = json.loads(temp_u.read().decode())
        edges = js_data['data']['user']['edge_owner_to_timeline_media']['edges']
        cursor = js_data['data']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        flag = js_data['data']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
        get_twelve_post(edges)
    else:
        break

data/nasa/  already exist
